In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp  /content/drive/MyDrive/kvasir_dataset.zip .

In [3]:
!unzip -qq kvasir_dataset.zip

In [4]:
!rm -rf kvasir_dataset/train/.ipynb_checkpoints

In [5]:
!pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [7]:
weight_decay = 0.0001
batch_size = 64
num_epochs = 50
dropout_rate = 0.1
image_size = 128  # We'll resize input images to this size.
patch_size = 8  # Size of the patches to be extracted from the input images.
num_patches = (image_size // patch_size) ** 2  # Size of the data array.
embedding_dim = 256  # Number of hidden units.
num_blocks = 6  # Number of blocks.

print(f"Image size: {image_size} X {image_size} = {image_size ** 2}")
print(f"Patch size: {patch_size} X {patch_size} = {patch_size ** 2} ")
print(f"Patches per image: {num_patches}")
print(f"Elements per patch (3 channels): {(patch_size ** 2) * 3}")

Image size: 128 X 128 = 16384
Patch size: 8 X 8 = 64 
Patches per image: 256
Elements per patch (3 channels): 192


In [8]:
imagePaths = "kvasir_dataset/train/"
test_path = "kvasir_dataset/test/"

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  imagePaths,
  #color_mode='grayscale',
  validation_split=0.15,
  subset="training",
  seed=123,
  image_size=(image_size, image_size),
  #label_mode="categorical",
  batch_size=batch_size)

Found 15300 files belonging to 6 classes.
Using 13005 files for training.


In [10]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  imagePaths, 
  #color_mode='grayscale',
  validation_split=0.15,
  subset="validation",
  seed=123,
  image_size=(image_size, image_size),
  #label_mode="categorical",
  batch_size=batch_size)

Found 15300 files belonging to 6 classes.
Using 2295 files for validation.


In [11]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_path,
  seed=123,
  image_size=(image_size, image_size),
  #label_mode="categorical",
  batch_size=batch_size)

Found 2700 files belonging to 6 classes.


In [12]:
num_classes = 6
input_shape = (128, 128, 3)

In [13]:
def build_classifier(blocks, positional_encoding=False):
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size, num_patches)(augmented)
    # Encode patches to generate a [batch_size, num_patches, embedding_dim] tensor.
    x = layers.Dense(units=embedding_dim)(patches)
    if positional_encoding:
        positions = tf.range(start=0, limit=num_patches, delta=1)
        position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=embedding_dim
        )(positions)
        x = x + position_embedding
    # Process x using the module blocks.
    x = blocks(x)
    # Apply global average pooling to generate a [batch_size, embedding_dim] representation tensor.
    representation = layers.GlobalAveragePooling1D()(x)
    # Apply dropout.
    representation = layers.Dropout(rate=dropout_rate)(representation)
    # Compute logits outputs.
    logits = layers.Dense(num_classes)(representation)
    # Create the Keras model.
    return keras.Model(inputs=inputs, outputs=logits)

In [14]:
def run_experiment(model):
    # Create Adam optimizer with weight decay.
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay,
    )
    # Compile the model.
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="acc"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top5-acc"),
        ],
    )
    # Create a learning rate scheduler callback.
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5
    )
    # Create an early stopping callback.
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=100, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        train_ds,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=val_ds,
        callbacks=[reduce_lr],
    )

    _, accuracy, top_5_accuracy = model.evaluate(test_ds)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    # Return history to plot learning curves.
    return history

In [15]:
from tensorflow.keras.models import Sequential

In [16]:
data_augmentation= Sequential([
                               #layers.Rescaling(scale=1.0 / 255),
                               layers.RandomBrightness(factor=0.2, value_range=(0, 255), seed=123)
])

In [17]:
class Patches(layers.Layer):
    def __init__(self, patch_size, num_patches):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = num_patches

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        return patches

In [18]:
class MLPMixerLayer(layers.Layer):
    def __init__(self, num_patches, hidden_units, dropout_rate, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.mlp1 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=num_patches),
                layers.Dropout(rate=dropout_rate),
            ]
        )
        self.mlp2 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=embedding_dim),
                layers.Dropout(rate=dropout_rate),
            ]
        )
        self.normalize = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        # Apply layer normalization.
        x = self.normalize(inputs)
        # Transpose inputs from [num_batches, num_patches, hidden_units] to [num_batches, hidden_units, num_patches].
        x_channels = tf.linalg.matrix_transpose(x)
        # Apply mlp1 on each channel independently.
        mlp1_outputs = self.mlp1(x_channels)
        # Transpose mlp1_outputs from [num_batches, hidden_dim, num_patches] to [num_batches, num_patches, hidden_units].
        mlp1_outputs = tf.linalg.matrix_transpose(mlp1_outputs)
        # Add skip connection.
        x = mlp1_outputs + inputs
        # Apply layer normalization.
        x_patches = self.normalize(x)
        # Apply mlp2 on each patch independtenly.
        mlp2_outputs = self.mlp2(x_patches)
        # Add skip connection.
        x = x + mlp2_outputs
        return x

In [ ]:
mlpmixer_blocks = keras.Sequential(
    [MLPMixerLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.05
mlpmixer_classifier = build_classifier(mlpmixer_blocks)
history = run_experiment(mlpmixer_classifier)

Epoch 1/50


204/204 [==============================] - 259s 1s/step - loss: 466.9145 - acc: 0.4457 - top5-acc: 0.9559 - val_loss: 112.9930 - val_acc: 0.3425 - val_top5-acc: 0.9490 - lr: 0.0500
Epoch 2/50
204/204 [==============================] - 253s 1s/step - loss: 1438.0935 - acc: 0.4501 - top5-acc: 0.9562 - val_loss: 233.8462 - val_acc: 0.5769 - val_top5-acc: 0.9978 - lr: 0.0500
Epoch 3/50
204/204 [==============================] - 245s 1s/step - loss: 63.3798 - acc: 0.6120 - top5-acc: 0.9984 - val_loss: 35.1178 - val_acc: 0.5403 - val_top5-acc: 0.9917 - lr: 0.0500
Epoch 4/50
204/204 [==============================] - 243s 1s/step - loss: 18.3113 - acc: 0.6341 - top5-acc: 0.9992 - val_loss: 16.4720 - val_acc: 0.6209 - val_top5-acc: 0.9983 - lr: 0.0500
Epoch 5/50
204/204 [==============================] - 250s 1s/step - loss: 11.2172 - acc: 0.6365 - top5-acc: 0.9990 - val_loss: 21.6301 - val_acc: 0.5473 - val_top5-acc: 0.9978 - lr: 0.0500
Epoch 6/50
204/204 [==============================] - 24

In [19]:
mlpmixer_blocks = keras.Sequential(
    [MLPMixerLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.05
mlpmixer_classifier = build_classifier(mlpmixer_blocks)
history = run_experiment(mlpmixer_classifier)

Epoch 1/50


204/204 [==============================] - 243s 1s/step - loss: 753.8687 - acc: 0.4190 - top5-acc: 0.9559 - val_loss: 77.0919 - val_acc: 0.2871 - val_top5-acc: 0.9569 - lr: 0.0500
Epoch 2/50
204/204 [==============================] - 226s 1s/step - loss: 135.5065 - acc: 0.5020 - top5-acc: 0.9762 - val_loss: 4064.2944 - val_acc: 0.2362 - val_top5-acc: 0.8370 - lr: 0.0500
Epoch 3/50
204/204 [==============================] - 224s 1s/step - loss: 534.3361 - acc: 0.5256 - top5-acc: 0.9830 - val_loss: 10.6375 - val_acc: 0.6993 - val_top5-acc: 0.9996 - lr: 0.0500
Epoch 4/50
204/204 [==============================] - 224s 1s/step - loss: 11.6115 - acc: 0.6336 - top5-acc: 0.9986 - val_loss: 7.3482 - val_acc: 0.6501 - val_top5-acc: 0.9969 - lr: 0.0500
Epoch 5/50
204/204 [==============================] - 224s 1s/step - loss: 7.4720 - acc: 0.6281 - top5-acc: 0.9978 - val_loss: 4.5964 - val_acc: 0.6754 - val_top5-acc: 1.0000 - lr: 0.0500
Epoch 6/50
204/204 [==============================] - 224s 

In [20]:
mlpmixer_classifier.evaluate(test_ds)

43/43 [==============================] - 30s 569ms/step - loss: 2.8807 - acc: 0.6304 - top5-acc: 0.9996


[2.8806607723236084, 0.6303703784942627, 0.9996296167373657]

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [26]:
predictions = np.array([])
labels =  np.array([])
for x, y in test_ds:
  Y_pred=mlpmixer_classifier.predict(x)
  y_prediction = np.argmax(Y_pred, axis=1)
  predictions = np.concatenate([predictions, y_prediction])
  labels = np.concatenate([labels, y.numpy()])

1/1 [==============================] - 0s 42ms/step


In [27]:
print('Confusion Matrix')
print(confusion_matrix(y_true=labels, y_pred=predictions))

Confusion Matrix
[[ 93 355   1   0   1   0]
 [ 41 406   0   2   1   0]
 [  0   1 371  77   1   0]
 [  8   5  31 397   8   1]
 [ 16  26   8  59 320  21]
 [  7   6   3  38 281 115]]


In [28]:
classes=['dyed-lifted-polyps','dyed-resection-margins', 'esophagitis','normal','polyps', 'ulcerative-colitis']
     
print('Classification Report')
target_names = classes
print(classification_report(y_true=labels, y_pred=predictions, target_names=target_names))

Classification Report
                        precision    recall  f1-score   support

    dyed-lifted-polyps       0.56      0.21      0.30       450
dyed-resection-margins       0.51      0.90      0.65       450
           esophagitis       0.90      0.82      0.86       450
                normal       0.69      0.88      0.78       450
                polyps       0.52      0.71      0.60       450
    ulcerative-colitis       0.84      0.26      0.39       450

              accuracy                           0.63      2700
             macro avg       0.67      0.63      0.60      2700
          weighted avg       0.67      0.63      0.60      2700



In [29]:
mlpmixer_classifier.save("mlpmixer_classifier_kvasir")

In [30]:
!zip -r mlpmixer_classifier_kvasir.zip mlpmixer_classifier_kvasir/

  adding: mlpmixer_classifier_kvasir/ (stored 0%)
  adding: mlpmixer_classifier_kvasir/saved_model.pb (deflated 92%)
  adding: mlpmixer_classifier_kvasir/keras_metadata.pb (deflated 95%)
  adding: mlpmixer_classifier_kvasir/variables/ (stored 0%)
  adding: mlpmixer_classifier_kvasir/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: mlpmixer_classifier_kvasir/variables/variables.index (deflated 79%)
  adding: mlpmixer_classifier_kvasir/assets/ (stored 0%)
